### Machine Learning Notebook

This notebook will implement the machine learning models needed to predict house sale price.  The following machine learning models including a lasso regression, ridge regression, elastic net regression, a random forest model, gradient boosted models, a stacked, and a blended model will be implemented.  The output training RMSE of the models will be compared and the model with the lowest error will predict on the test set.  The resulting predictions of the best model will be submitted to Kaggle.

The Koalas library was used when interacting with big data because it implements the Pandas DataFrame API on top of Apache Spark. Pandas is typically used for single node Dataframe implementation in Python while Spark is used for big data processing. With Koalas, Pandas syntax can be used while processing dataframes with Apache Spark.

In [4]:
dbutils.library.installPyPI("koalas")
dbutils.library.restartPython()

The model will be evaluated on RMSE which is the Root Mean Squared Error between the prediction and the label.  When the dataframe is loaded, some of the columns that are numeric should be strings as string format is more accurate for values that are not ordered or increasing in value.  An extra field was created called YrBltAndRemod that is the sum of the YearBuilt and the YearRemodAdd features.

In [6]:
import databricks.koalas as ks
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id
import numpy as np
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p
from pyspark.ml.feature import StandardScaler
from sklearn.preprocessing import RobustScaler

full_df=pd.read_csv('/dbfs/FileStore/cleaned1.csv', index_col=0)
train_label=pd.read_csv('/dbfs/FileStore/trainlabel.csv', header=None)

full_df['MSSubClass']=full_df['MSSubClass'].astype(str)
full_df['MoSold']=full_df['MoSold'].astype(str)
full_df['YrSold']=full_df['YrSold'].astype(str)

full_df['YrBltAndRemod']=full_df['YearBuilt']+full_df['YearRemodAdd']

The numeric features were collected so that they could be transformed to normal distributions.  A normal distrubtion helps a machine learning model evaluate the samples because outliers will not be as unevely distributed once the feature  is transformed to a normal distribution.  If the value of skew was above 0.5, the boxcox1p transformation was applied.  The transformaion is involves taking the input value to exponent of the optimal lambda, subtracting one, and then dividing by lambda.  This occurs when lambda is not 0 and when lambda is equal to 0, the log of the values of the feature are found.

In [8]:
numeric_features=[]
for i in full_df.columns:
    if full_df[i].dtype!=object:
        numeric_features.append(i)


    
    
skew_features=full_df[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew=skew_features=skew_features[skew_features>0.5]
skew_index=high_skew.index

for i in skew_index:
  full_df[i]=boxcox1p(full_df[i], boxcox_normmax(full_df[i]+1))

  
scaler = RobustScaler()

full_df[numeric_features] = scaler.fit_transform(full_df[numeric_features])


/databricks/python/lib/python3.7/site-packages/scipy/stats/stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
 r = r_num / r_den

The full dataset was split into train and test sets.  The train data was subsetted to only include samples with a GrLivArea less than 4500 as whese values were found to be outliders.  Becuase the models will be evaulation on difference of the log RMSE and labels, the labels were transformed by taking the log of the values.

In [10]:
train_df=full_df[:1460]
test_df=full_df[1460:]


train_df = train_df[train_df.GrLivArea < 4500]

train_df['label']=np.log1p(train_label[1])




Spark dataframes were createad from the test, train and full dataframes.

In [12]:
X=train_df
X_test=test_df
X_dropped=X.drop(['label'], axis=1)

X=pd.concat([X_dropped, X_test])
X=pd.get_dummies(X)

X_test=X[1460:]
X=X[:1460]


y=np.log1p(train_label[1])


train_df = spark.createDataFrame(train_df)
full_df=spark.createDataFrame(full_df)
test_df = spark.createDataFrame(test_df)

print((full_df.count(), len(full_df.columns)))
print((train_df.count(), len(train_df.columns)))
print((test_df.count(), len(test_df.columns)))


(2919, 90)
(1460, 91)
(1459, 90)

We will have to convert the categorical variables in the dataset into numeric variables. There are 2 steps to do this.


The below code basically indexes each categorical column using the StringIndexer, and then converts the indexed categories into one-hot encoded variables. The resulting output has the binary vectors appended to the end of each row.

In [14]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator
from pyspark.sql.types import *


categoricalColumns=[]
for i in full_df.columns:
  if isinstance(full_df.schema[i].dataType, StringType):
    categoricalColumns.append(i)

stages=[]  #stages in our Pipeline
for categoricalCol in categoricalColumns:
  stringIndexer=StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  
  encoder=OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol+"classVec"], dropLast=False)
  
  stages += [stringIndexer, encoder]
  



Group together numerical columns and then use a VectorAssembler to combine all the feature columns, categorical and numerica, into a single vector column. This includes both the numeric columns and the one-hot encoded binary vector columns in our dataset.

In [16]:
numericCols=[]
for i in full_df.columns:
  if i=="label":
    continue
  if isinstance(full_df.schema[i].dataType, StringType)==False:
    numericCols.append(i)
   
    

assemblerInputs=[c + "classVec" for c in categoricalColumns] + numericCols
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

Run the stages as a Pipeline. This puts the data through all of the feature transformations we described in a single call.

In [18]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel

partialPipeline = Pipeline().setStages(stages)
pipelineModel=partialPipeline.fit(full_df)
preppedDataDF=pipelineModel.transform(train_df)





The test set is simlilary Run through the stages as a Pipeline. This puts the data through all of the feature transformations we described in a single call.

In [20]:
partialPipeline = Pipeline().setStages(stages)
pipelineModel=partialPipeline.fit(full_df)
preppedDataDF_test=pipelineModel.transform(test_df)




The optimal hyperparameters are found using the hyperopt package which implements bayesian optimizaiton.  

Bayesian optimizaiton can be explained in four steps:
1.) Initialize process by sampling the hyperparameter space either randomly or low-discrepancy sequencing and getting these observations. 

2.) In this case, fit a Gaussian process to the observed data from step 1. Use the mean from the Gaussian process as the function most likely to model the black box function.

3.) Use the maximal location of the acquisition function to figure out where to sample next in the hyperparameter space. 

4.) Get an observation of the black box function given the newly sampled hyperparameter points. Add observations to the set of observed data.


This process (Steps 2-4) repeats until a maximum number of iterations is met. By iterating through the method explicated above, Bayesian optimization effectively searches the hyperparameter space while homing in on the global optima.

In [22]:
!pip install hyperopt
from hyperopt import fmin, hp, tpe
from hyperopt import SparkTrials, STATUS_OK
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.model_selection import KFold, cross_val_score
from functools import partial

search_space_ridge = {
  'alphas': hp.choice('alphas', [14.5, 15, 15.1, 15.2, 15.3, 15.4]),
  'max_iter': hp.choice('max_iter', [1e7])
}

search_space_lasso = {
  'alphas': hp.choice('alphas', [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]),
  'max_iter': hp.choice('max_iter', [1e7])

}

search_space_elastic = {
  'alphas': hp.choice('alphas', [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]),
  'max_iter': hp.choice('max_iter', [1e7]),
  'l1_ratio': hp.choice('l1_ratio', [1])
}

algo=partial(tpe.suggest, n_startup_jobs=1, n_EI_candidates=1)

rstate = np.random.RandomState(42)

def cv_rmse(model, X=X):
    kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

def train_ridge(params):
  clf=RidgeCV(cv=10)
  score = cv_rmse(clf).mean()
  return {'loss': score, 'status': STATUS_OK}

def train_lasso(params):
  clf=LassoCV(cv=10)
  score = cv_rmse(clf).mean()
  return {'loss': score, 'status': STATUS_OK}

def train_elastic(params):
  clf=ElasticNetCV(cv=10)
  score = cv_rmse(clf).mean()
  return {'loss': score, 'status': STATUS_OK}


best_hyperparameters_ridge = fmin(
        fn=train_ridge,
        space=search_space_ridge,
        algo=algo,
        max_evals=10,
        return_argmin=False,
        )

print('Ridge HyperParameters: ' + str(best_hyperparameters_ridge))
'''Ridge HyperParameters: {'alphas': 14.5, 'max_iter': 10000000.0}'''


best_hyperparameters_lasso = fmin(
  fn=train_lasso,
  space=search_space_lasso,
  algo=algo,
  max_evals=12,
  return_argmin=False
  )

print('Lasso HyperParameters: ' + str(best_hyperparameters_lasso))


'''Lasso HyperParameters: {'alphas': 0.0008, 'max_iter': 10000000.0}'''

best_hyperparameters_elastic = fmin(
  fn=train_elastic,
  space=search_space_elastic,
  algo=algo,
  max_evals=20,
  
  return_argmin=False
  )


print('Elastic HyperParameters: ' + str(best_hyperparameters_elastic))
'''Elastic HyperParameters: {'alphas': 0.0007, 'l1_ratio': 1, 'max_iter': 10000000.0}'''

Now that the optimal hyperparameters from bayesian optimization were found, we can build small hyperparameter grids with only one set of hyperparameters that can be searched quickly, which will lower training times.  


Linear regression predicts the response variable in this case home sale price as the dot product of p distinct predictors and their corresponding coefficients coefficients.  An intercept term is added to the dot product so the response variable has a non zero value when all of the preditor values are zero.  In this case home sale price should have an intercept term because home price must be greater than 0.  The multiple linear regression model takes the form
Y = β0 + β1X1 + β2X2 + ··· + βpXp + ... βj can be interpreted as the increase in the response variable with one unit increase of Xj assuming all other slope coefficients are set to zero.  


Three different estimators were tested: ridge regression, lasso regression, and elastic net regression.  These three types of regression include different regularization types. In the final report, the equations for the types of regression with regularization will be listed.  The purpose of regularization is to prevent overfitting by adding penalty terms that cause the loss function to have less predictor terms. 


In order to implement Ridge regression, the elastic parameter must be set to zero which cancels out the L1 term.  To implement lasso regression, the elastic parameter must be 1 to cancel out the L2 term.  To implement elastic net, the elastic net parameter as well as the L1 term must be greater than 0 and less than 1 but 1 was the optimal value.

In [24]:

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel 
from pyspark.ml.evaluation import RegressionEvaluator

regEval = RegressionEvaluator(predictionCol="prediction", 
                 labelCol="label", metricName = "rmse")

lr=LinearRegression(labelCol="label", featuresCol="features", predictionCol="prediction")

# Create ParamGrid for Cross Validation

#ridge

paramGrid_ridge = (ParamGridBuilder()
             .addGrid(lr.regParam, [14.5])
             .addGrid(lr.elasticNetParam, [0])
             .addGrid(lr.maxIter, [1e7])
             .build())

#lasso
paramGrid_lasso = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.008])
             .addGrid(lr.elasticNetParam, [1])
             .addGrid(lr.maxIter, [1e7])
             .build())

#elastic net

paramGrid_elastic = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.0007])
             .addGrid(lr.elasticNetParam, [1])
             .addGrid(lr.maxIter, [1e7])
             .build())



#elasticnet


Cross validation was performed on the training set with 10 folds and evaluated using the RMSE.  The fitted models were saved in order to save time so that they do not need to be rerun.

In [26]:
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid_ridge, evaluator=regEval, numFolds=10)
cvModel_ridge_ = cv.fit(preppedDataDF)

cvModel_ridge_.write().overwrite().save('myModel_ridge')
cvModel_ridge=CrossValidatorModel.load('myModel_ridge')


cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid_lasso, evaluator=regEval, numFolds=10)
cvModel_lasso_ = cv.fit(preppedDataDF)

cvModel_lasso_.write().overwrite().save('myModel_lasso')
cvModel_lasso=CrossValidatorModel.load('myModel_lasso')

cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid_elastic, evaluator=regEval, numFolds=10)
cvModel_elastic_ = cv.fit(preppedDataDF)


cvModel_elastic_.write().overwrite().save('myModel_elastic')
cvModel_elastic=CrossValidatorModel.load('myModel_elastic')


The following metrics were outputted of the model: The intercept of the linear regression function, the top 15 features with the highest coefficients, the optimal parameters of the model, the number of iterations the model took, the best training RMSE, the r^2 value of the linear regression fit, and the RMSE of the validation set for the best parameter combination.  The largest coefficients tell us what can be changed about the house to increase the house sale price.


The elastic net model recieved a validation score of 0.1284 and the ridge regression model recieved a validation score of 0.2808.  The optimal hyperparameters in the elastic net model removed the L2 term which means Lasso  must have performed better than ridge regression.    The lasso regression model performed the best with a validation RMSE of 0.1250.

In [28]:

def metrics(cvModel, model):

  coeff=cvModel.bestModel.coefficients
  print("Intercept: %s" % str(cvModel.bestModel.intercept))
  # Summarize the model over the training set and print out some metrics
  feature_names=(preppedDataDF.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+preppedDataDF.schema["features"].metadata["ml_attr"]["attrs"]["numeric"])

  features=[]
  for i in range(len(feature_names)):
    features.append(feature_names[i]['name'])

  coeffs=list(zip(features, coeff))
  coeffs = sorted(coeffs, key=lambda tup: tup[1], reverse=True)[0:15]
  print(coeffs)

  #trainingSummary = cvModel_lasso_.bestModel.summary

  best_reg_param = cvModel.bestModel._java_obj.getRegParam()
  print('Regularization Parameter: ' + str(best_reg_param))
  if cvModel==cvModel_elastic:
    best_elastic_param = cvModel.bestModel._java_obj.getElasticNetParam()
    print('ElasticNet Parameter: ' + str(best_elastic_param))
  
  
  trainingSummary = model.bestModel.summary
  print("numIterations: %d" % trainingSummary.totalIterations)
  print('RMSE: {}'.format(trainingSummary.rootMeanSquaredError))
  print("r2: %f" % trainingSummary.r2)
  print('Average Metrics for each Parameter Combination: ' + str(model.avgMetrics))
  print('\n\n')

metrics(cvModel_ridge, cvModel_ridge_) 
metrics(cvModel_lasso, cvModel_lasso_)
metrics(cvModel_elastic, cvModel_elastic_) 




'''Intercept: 11.977677521849378
Top Features: [('PoolQCclassVec_Ex', 0.019000667896077787), ('RoofMatlclassVec_WdShngl', 0.01602273617277807), ('Condition2classVec_PosA', 0.01543805075340896), ('ExterQualclassVec_Ex', 0.01421741838657165), ('NeighborhoodclassVec_NoRidge', 0.012666748145105981), ('FireplaceQuclassVec_Ex', 0.012527127269855954), ('KitchenQualclassVec_Ex', 0.01235360560865593), ('BsmtQualclassVec_Ex', 0.012217563394786614), ('Exterior2ndclassVec_Other', 0.011434343474303024), ('NeighborhoodclassVec_NridgHt', 0.01131313661231901), ('NeighborhoodclassVec_StoneBr', 0.011026366215851235), ('CentralAirclassVec_Y', 0.009440796647356895), ('SaleTypeclassVec_Con', 0.009303717513164412), ('Exterior1stclassVec_Stone', 0.009264673590399534), ('hasgarage', 0.009208381063546765)]
Regularization Parameter: 14.5
numIterations: 1
RMSE: 0.27948058559447003
r2: 0.510134
Average Validation RMSE for best Parameter Combination: [0.2808121355653476]



Intercept: 11.963706867662323
Top Features: [('Total_SF', 0.1059140989389088), ('GrLivArea', 0.08357871462008833), ('Total_Home_Quality', 0.07938970656123005), ('NeighborhoodclassVec_Crawfor', 0.07244816040350494), ('YrBltAndRemod', 0.07063186115311705), ('NeighborhoodclassVec_StoneBr', 0.05892897856487002), ('KitchenQualclassVec_Ex', 0.055302642687728096), ('BsmtQualclassVec_Ex', 0.0497972409120521), ('NeighborhoodclassVec_NoRidge', 0.04283947669470563), ('Exterior1stclassVec_BrkFace', 0.041378198106361515), ('BsmtFinSF1', 0.04072661981812749), ('FunctionalclassVec_Typ', 0.03843061897704698), ('OverallQual', 0.03716151956168724), ('GarageCars', 0.034263181390235195), ('SaleTypeclassVec_New', 0.03384804680101499)]
Regularization Parameter: 0.008
numIterations: 294
RMSE: 0.1088871197501967
r2: 0.925642
Average Validation RMSE for best Parameter Combination: [0.125056342529829]



Intercept: 11.813980776889892
Top Features: [('GarageQualclassVec_Ex', 0.24681017986441736), ('RoofMatlclassVec_Membran', 0.23057278879163012), ('Condition2classVec_PosA', 0.1735019582510545), ('PoolQCclassVec_Gd', 0.14835817467259307), ('RoofStyleclassVec_Shed', 0.14083215165847704), ('NeighborhoodclassVec_StoneBr', 0.13688117779597792), ('Total_SF', 0.11829057573914872), ('NeighborhoodclassVec_Crawfor', 0.11157239563318498), ('RoofMatlclassVec_Metal', 0.11113008836938372), ('RoofMatlclassVec_WdShngl', 0.1016673278090774), ('PoolQCclassVec_Ex', 0.09944854240169966), ('SaleTypeclassVec_ConLD', 0.09016653084130574), ('GrLivArea', 0.08190836070021754), ('NeighborhoodclassVec_NoRidge', 0.08040881506696701), ('Exterior1stclassVec_BrkFace', 0.07814012472224557)]
Regularization Parameter: 0.0007
ElasticNet Parameter: 1.0
numIterations: 1224
RMSE: 0.09158737568716792
r2: 0.947393
Average Validation RMSE for Best Parameter Combination: [0.12843334246807203]

'''
  
  


The random forest classifier is an ensemble method that uses many decision trees that have had bootstrap aggregating or bagging done on the samples. Bagging means that samples are chosen at random with replacement to reduce variance in any single tree and the results are averaged. The prediction in an individual tree is the average value of the response variable for the samples of a leaf node.  The prediction of the bagged regression decision tree  is the average of the predictions for each individual decision tree.  

Random forests in addition take a subset of the predictors when constructing a tree so that each tree is more different from one another. The combination of the trees made from bagged samples and subsetted predictors is the random forest result. The random forest model was the worst performing ensemble method with a validation RMSE of 0.1682.

In [30]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel 
from pyspark.ml import Pipeline


rf = RandomForestRegressor(labelCol="label", featuresCol="features", predictionCol="prediction")

regEval = RegressionEvaluator(predictionCol="prediction", 
                 labelCol="label", metricName = "rmse")


paramGrid_rf = (ParamGridBuilder()
             .addGrid(rf.featureSubsetStrategy, ['sqrt'])
             .addGrid(rf.maxDepth, [5])
             .addGrid(rf.numTrees, [3000])
             .build())

cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid_rf, evaluator=regEval, numFolds=10)

pipeline=Pipeline(stages=[cv])
cvModel_rf_ = cv.fit(preppedDataDF)



In [31]:
cvModel_rf_.write().overwrite().save('myModel_rf')
cvModel_rf=CrossValidatorModel.load('myModel_rf')

In [32]:
def metrics(cvModel, model):

  feat_imp=cvModel.bestModel.featureImportances

  feature_names=(preppedDataDF.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+preppedDataDF.schema["features"].metadata["ml_attr"]["attrs"]["numeric"])

  features=[]
  for i in range(len(feature_names)):
    features.append(feature_names[i]['name'])

  feat_imp=list(zip(features, feat_imp))
  feat_imp = sorted(feat_imp, key=lambda tup: tup[1], reverse=True)[0:15]
  print(feat_imp)
  
  print('Average Validation RMSE for Best Parameter Combination: ' + str(model.avgMetrics))

  
metrics(cvModel_rf, cvModel_rf_) 


'''Most Important features: [('Total_SF', 0.07942801665560263), ('OverallQual', 0.0762820067958877), ('GrLivArea', 0.05705986955640432), ('Total_bath', 0.03945939278404038), ('YearBuilt', 0.03886143544353758), ('YrBltAndRemod', 0.03878246729733818), ('GarageCars', 0.03732368466637657), ('Total_Home_Quality', 0.03413133362982473), ('ExterQualclassVec_TA', 0.03300783939600958), ('GarageArea', 0.03262434890655314), ('TotalBsmtSF', 0.031826775360823144), ('FullBath', 0.02902317536062917), ('1stFlrSF', 0.026346867027809858), ('GarageYrBlt', 0.024781569363908476), ('YearRemodAdd', 0.02113836596814282)]
Average Validation RMSE for Best Parameter Combination: [0.16819963092531479]'''

A gradient boosted classifier involves combining a large number of trees with bagging and boosting. Each subsequent tree that is constructed is done so based on the least pure splits created by the previous tree. The trees are added back to the previous tree so that the tree can learn slowly and improve RMSE. An advantage to gradient boosted trees is that compared to a decision tree that may overfit to one set of the data because gradient boosted trees learn by focusing on the worst split in a tree and improve over time. The disadvantages of gradient boosted trees is that they may overfit compared to random forests but not in this case.


The gradient boosted classifier had an average valdiation RMSE of 0.1208 on the training set.

In [34]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV


def cv_rmse(model, X=X):
    kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)


clf=GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)      
score = cv_rmse(clf).mean()
print('Average Validation Loss: ' + str(score))

print('\033[1m' + 'Top 10 features in gradient boosted tree that contribute to home sale price:' + '\033[0m')

gsc = GridSearchCV(
        estimator=clf,
        param_grid={}, cv=10, scoring='neg_mean_squared_error')



gbt=gsc.fit(X, y)

y_pred_gbr=gbt.predict(X_test)

print('Top 10 features in gradient boosted tree that contribute to home sale price:')
sorted(list(zip(X.columns, gbt.best_estimator_.feature_importances_)), key=lambda x: x[1], reverse=True)[:10]


'''Average Validation Loss: 0.12079922229921425
Top 10 features in gradient boosted tree that contribute to home sale price:
Top 10 features in gradient boosted tree that contribute to home sale price:
Out[19]: [('GrLivArea', 0.13141904142177038),
 ('Total_SF', 0.08782575198710837),
 ('OverallQual', 0.06680771059701276),
 ('TotalBsmtSF', 0.06581417875915589),
 ('YrBltAndRemod', 0.04300053789530215),
 ('GarageArea', 0.04141307756162639),
 ('LotArea', 0.03350685544843198),
 ('Fireplaces', 0.03295994764788792),
 ('ExterQual_Gd', 0.029057445867734944),
 ('Total_Home_Quality', 0.027583003487325338)]'''

The lightgbm is a type of gradient boosting regressors in that new trees are learning from the worst splits in the previous tree.  One key difference is that lightgbm regressor grow leaf wise as opposed to level wise of gradient boosting regressors.  Leaf wise growth constructs a new tree from a leaf new and keeps creating new trees from the leaf nodes of the previously created trees.  Level wise growth must have all the leaves on a level of a tree split into new nodes between the resulting nodes can be split.  

Both lighgbm and xgboost use histogram based bins to split the data of an attribute into bins.  The decision tree only needs to check the number of bins for each attribute.  

The lightgbm model achieved an average validation loss 0.1207.

In [36]:
!pip install lightgbm
from sklearn.model_selection import KFold, cross_val_score
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV


def cv_rmse(model, X=X):
    kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)


clf=LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )   

score = cv_rmse(clf).mean()
print('Average Validation Loss: ' + str(score))

print('\033[1m' + 'Top 10 features in light gradient boosted tree that contribute to home sale price:' + '\033[0m')

gsc = GridSearchCV(
        estimator=clf,
        param_grid={}, cv=10, scoring='neg_mean_squared_error')



gbt=gsc.fit(X, y)

y_pred_gbr=gbt.predict(X_test)

print('Top 10 features in gradient boosted tree that contribute to home sale price:')
sorted(list(zip(X.columns, gbt.best_estimator_.feature_importances_)), key=lambda x: x[1], reverse=True)[:10]

'''Average Validation Loss: 0.12072925676713671
Top 10 features in light gradient boosted tree that contribute to home sale price:
Top 10 features in gradient boosted tree that contribute to home sale price:
Out[20]: [('LotArea', 651),
 ('GrLivArea', 598),
 ('Total_SF', 594),
 ('1stFlrSF', 572),
 ('GarageArea', 501),
 ('TotalBsmtSF', 487),
 ('Total_porch_sf', 463),
 ('LotFrontage', 445),
 ('YrBltAndRemod', 425),
 ('YearBuilt', 402)]'''

The xgboost is a type of gradient boosting regressors in that new trees are learning from the worst splits in the previous tree.  One key difference is that xgboost regressor grow leaf wise as opposed to level wise of gradient boosting regressors.  Leaf wise growth constructs a new tree from a leaf new and keeps creating new trees from the leaf nodes of the previously created trees.  Level wise growth must have all the leaves on a level of a tree split into new nodes between the resulting nodes can be split.  

Both lighgbm and xgboost use histogram based bins to split the data of an attribute into bins.  The decision tree only needs to check the number of bins for each attribute.  XGBoost is also known as the regularised version of GBM. This framework includes built-in L1 and L2 regularisation which means it can prevent a model from overfitting.

The xgboost model achieved an average validation loss was 0.1166.

In [38]:
!pip install xgboost
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV


def cv_rmse(model, X=X):
    kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

clf=XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

score = cv_rmse(clf).mean()
print('Average Validation Loss: ' + str(score))

gbt = GridSearchCV(
        estimator=clf,
        param_grid={}, cv=10, scoring='neg_mean_squared_error')

gbt.fit(X, y)

y_pred_xboost=gbt.predict(X_test)

#sorted(list(zip(X.columns, gbt.best_estimator_.feature_importances_)), key=lambda x: x[1], reverse=True)[:10]

'''Average Validation Loss: 0.11659982710488989
Top 10 features in xg-boosted tree that contribute to home sale price:
Out[21]: [('GarageCars', 0.053317003),
 ('hasfireplace', 0.052065946),
 ('Total_SF', 0.050921544),
 ('Total_bath', 0.034195196),
 ('BsmtQual_Ex', 0.03344998),
 ('OverallQual', 0.03156224),
 ('Fireplaces', 0.03149775),
 ('YrBltAndRemod', 0.031276856),
 ('CentralAir_Y', 0.024429448),
 ('Total_Home_Quality', 0.022839572)]'''

Stacking enables a method of ensembling multiple regression models. 

The training data is split into K-folds and a base model is fitted on the K-1 parts.  A prediction is made for the Kth part and repeated for each part of the train set.  The base model is then fitted on the whole train dataset.  The previous steps are repeated for all of the base models.  The predictions of the base models from the k-fold cross validation of the train set are used as features in the meta-regressor model.  This second level model is used to make final predictions on the test set which has added features from the prediction made from the fitted base model predictions on the test set.

As a side note, blending is similar to stacking bt uses a training and validation split instead of k-fold cross validation.  The average validaion RMSE of the model was 0.1195.

In [40]:
!pip install mlxtend
!pip install lightgbm
!pip install xgboost
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline


kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

  
def cv_rmse(model, X=X):

    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)
  
ridge = RidgeCV(alphas=[14.5], cv=kfolds)
lasso = LassoCV(max_iter=1e7, alphas=[0.0008], random_state=42, cv=kfolds)

elasticnet = ElasticNetCV(max_iter=1e7, alphas=[0.0007],
                                        cv=kfolds, l1_ratio=[1])
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
                                   
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
                                       
xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)


# stack
stack_gen = StackingCVRegressor(regressors=(lasso, lightgbm, xgboost, gbr, elasticnet, lasso),
                                meta_regressor=lightgbm,
                                use_features_in_secondary=True)



''''stack = GridSearchCV(
       estimator=stack_gen, param_grid={},
        cv=10, scoring='neg_mean_squared_error')


stack.fit(X, y)
Stack score with lightgbm: 0.1195, 
y_pred_xboost=stack.predict(X_test)'''

Out[8]: "'stack = GridSearchCV(\n estimator=stack_gen, param_grid={},\n cv=10, scoring='neg_mean_squared_error')\n\n\nstack.fit(X, y)\nStack score with lightgbm: 0.1195, \ny_pred_xboost=stack.predict(X_test)"

A weighted average of the best performing models was found which resulted in the best validation RMSE of 0.073.

In [42]:
print('START Fit')

print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))

print('elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)

print('Lasso')
lasso_model_full_data = lasso.fit(X, y)

##### fix weights

print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)

print('xgboost')
xgb_model_full_data = xgboost.fit(X, y)

print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

START Fit
stack_gen
elasticnet
Lasso
GradientBoosting
xgboost
lightgbm

In [43]:
from sklearn.metrics import mean_squared_error

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def blend_models_predict(X):
    return ((((1/6) * elastic_model_full_data.predict(X)) + 
            ((1/6) * lasso_model_full_data.predict(X)) + 
            ((1/6) * gbr_model_full_data.predict(X)) + 
            ((1/6) * xgb_model_full_data.predict(X)) + 
            ((1/6) * lgb_model_full_data.predict(X)) + 
            ((1/6)* stack_gen_model.predict(np.array(X)))))
  
  

print('RMSLE score on train data:')
blend=(rmsle(y, blend_models_predict(X)))
print(blend)

RMSLE score on train data:
0.07318618629769486

### Model Metrics

In [45]:
print('\033[1m' + 'Regression Metrics for ML Models \n')
d2 = {'Validation RMSE': [0.1251, 0.2808, 0.1284, 0.1682, 0.1208, 0.1207, 0.1166, 0.1195, 0.0740], 'Test RMSE': [0.1256, 0.5067, 0.1276, 0.1734, 0.1287, 0.1273, 0.1229, 0.1221, 0.1194]}
index2=['Lasso regression', 'Ridge Regression', 'Elastic Net Regression', 'Random Forest', 'Gradient Boosted Classifier', 'XGBoost', 'LightGBM', 'Stacked Models', 'Weighted Avg Model']                

df2 = pd.DataFrame(data=d2,  index=index2)


df2

,Validation RMSE,Test RMSE
Lasso regression,0.1251,0.1256
Ridge Regression,0.2808,0.5067
Elastic Net Regression,0.1284,0.1276
Random Forest,0.1682,0.1734
Gradient Boosted Classifier,0.1208,0.1287
XGBoost,0.1207,0.1273
LightGBM,0.1166,0.1229
Stacked Models,0.1195,0.1221
Weighted Avg Model,0.0740,0.1194


In [46]:

print('Predict submission')
submission = pd.read_csv("/dbfs/FileStore/tables/sample_submission.csv")

submission.iloc[:,1] = np.floor(np.expm1(blend_models_predict(X_test)))
submission_avg=spark.createDataFrame(submission)
display(submission_avg)



Id,SalePrice
1461,122889.0
1462,159489.0
1463,184760.0
1464,198275.0
1465,192850.0
1466,171886.0
1467,180091.0
1468,165800.0
1469,188009.0
1470,122812.0
